


#import relevant packages




In [2]:
!pip install transformers datasets rouge_score bert-score
!pip install nltk
import pandas as pd
from datasets import load_dataset
from datasets import load_metric
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM
from typing import Dict, Any


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu1

#load dataset

In [ ]:
dataset = load_dataset('ccdv/pubmed-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

#choose 125 data from test

In [ ]:
data=dataset['test'].shuffle(seed=42).select(range(125))

#remove newline

In [ ]:
def remove_newlines(example):
    example['article'] = example['article'].replace('\n', '')
    return example
data = data.map(remove_newlines)

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

#use pretrained distil-bart model

In [ ]:
model_name = "sshleifer/distilbart-cnn-12-6"
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

#generate prediction

In [ ]:
def generate_summary(batch):
    inputs = tokenizer(batch["article"], return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    batch["pred"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return batch

In [ ]:
result = data.map(generate_summary)

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

KeyboardInterrupt: 

#compute and simplify rouge score

In [ ]:
rouge = load_metric("rouge")

<ipython-input-7-10357efb0a5b>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
rouge_scores = rouge.compute(predictions=result["pred"], references=data["abstract"])

{'rouge1': AggregateScore(low=Score(precision=0.4374014933950457, recall=0.17993581485925808, fmeasure=0.24466921408796957), mid=Score(precision=0.4614742770005783, recall=0.19416017143877928, fmeasure=0.2592634373329944), high=Score(precision=0.48612830242957905, recall=0.2077364982358767, fmeasure=0.272113777198652)), 'rouge2': AggregateScore(low=Score(precision=0.13144903306180425, recall=0.053606826508738645, fmeasure=0.07300401597512651), mid=Score(precision=0.1523044572212726, recall=0.06274633810447788, fmeasure=0.08465662272421048), high=Score(precision=0.1766682915704522, recall=0.07289163036687271, fmeasure=0.09750669926406395)), 'rougeL': AggregateScore(low=Score(precision=0.26568570638251704, recall=0.10953392121963772, fmeasure=0.1490634552997885), mid=Score(precision=0.2867928930629011, recall=0.11972126865949856, fmeasure=0.1600984470346707), high=Score(precision=0.3096687501704389, recall=0.13022481758150398, fmeasure=0.17203039639978354)), 'rougeLsum': AggregateScore(l

In [ ]:
def simplify_rouge_scores(rouge_scores: Dict[str, Any]) -> str:
    simplified_text = ""
    for key, value in rouge_scores.items():
        # Extract low, mid, and high scores for each ROUGE metric
        low, mid, high = value.low, value.mid, value.high
        simplified_text += f"{key}: Precision ranges from {low.precision:.2%} to {high.precision:.2%}, "
        simplified_text += f"Recall ranges from {low.recall:.2%} to {high.recall:.2%}, "
        simplified_text += f"F1 Score ranges from {low.fmeasure:.2%} to {high.fmeasure:.2%}.\n"

    return simplified_text

In [ ]:
print(simplify_rouge_scores(rouge_scores))

rouge1: Precision ranges from 43.74% to 48.61%, Recall ranges from 17.99% to 20.77%, F1 Score ranges from 24.47% to 27.21%.
rouge2: Precision ranges from 13.14% to 17.67%, Recall ranges from 5.36% to 7.29%, F1 Score ranges from 7.30% to 9.75%.
rougeL: Precision ranges from 26.57% to 30.97%, Recall ranges from 10.95% to 13.02%, F1 Score ranges from 14.91% to 17.20%.
rougeLsum: Precision ranges from 36.09% to 41.04%, Recall ranges from 14.91% to 17.22%, F1 Score ranges from 20.32% to 22.91%.



#output data file

In [ ]:
df = pd.DataFrame(result)
df.rename(columns={'pred': 'prediction'}, inplace=True)

In [ ]:
def add_newlines(row):
    row['article'] = row['article'].replace('. ', '.\n')
    row['prediction'] = row['prediction'].replace('. ', '.\n')
    return row
df = df.apply(add_newlines, axis=1)

In [ ]:
df.to_excel('bart_summary.xlsx', index=False)